# Simple Debug Notebook: MLFlow Tracing for Claude MCP Agent

This is a simplified version that avoids problematic imports and focuses on core tracing functionality.

In [1]:
# Simple, robust imports
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Dict, Any, List
import tempfile
import warnings
warnings.filterwarnings('ignore')

# Core MLFlow
import mlflow
from mlflow import MlflowClient

# Our agent
from dotenv import load_dotenv
from agent import create_agent

load_dotenv()
print("✅ All imports successful")

✅ All imports successful


In [2]:
# Simple MLFlow setup
mlflow_dir = os.path.abspath(os.getenv("MLFLOW_DIR", tempfile.mkdtemp(prefix="mlflow_debug_")))
mlflow_uri = f"file://{mlflow_dir}"

mlflow.set_tracking_uri(mlflow_uri)
experiment_name = "claude_mcp_debug_simple"
mlflow.set_experiment(experiment_name)

print(f"✅ MLFlow setup complete")
print(f"📁 Tracking URI: {mlflow_uri}")
print(f"🧪 Experiment: {experiment_name}")

Traceback (most recent call last):
  File "/Users/corvi42/Projects/wfd/venv/lib/python3.11/site-packages/mlflow/store/tracking/file_store.py", line 366, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/corvi42/Projects/wfd/venv/lib/python3.11/site-packages/mlflow/store/tracking/file_store.py", line 464, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/corvi42/Projects/wfd/venv/lib/python3.11/site-packages/mlflow/store/tracking/file_store.py", line 1634, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/corvi42/Projects/wfd/venv/lib/python3.11/site-packages/mlflow/store/tracking/file_store.py", line 1627, in _read_helper
    result = read_yaml(r

✅ MLFlow setup complete
📁 Tracking URI: file:///Users/corvi42/Projects/wfd/mlflow_data
🧪 Experiment: claude_mcp_debug_simple


In [3]:
# Create agent
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
MCP_SERVER_URL = "http://0.0.0.0:3000/mcp"
MODEL_ID = "claude-3-5-sonnet-20241022"

print("Creating agent...")
agent = create_agent(ANTHROPIC_API_KEY, MCP_SERVER_URL, MODEL_ID)
print(f"✅ Agent created with {len(agent.get_available_tools())} tools")
print(f"Available tools: {agent.get_available_tools()}")

Creating agent...
Initialized with session ID: 11deeae8-0c5e-46cc-bc0a-2b5a923795d7
Server info: {'name': 'restaurant-booking-server', 'version': '1.0.0'}
Available tools: ['search_restaurants', 'get_restaurant_details', 'get_booking_instructions', 'check_availability', 'make_reservation']
MCP session initialized successfully!
Available tools: ['search_restaurants', 'get_restaurant_details', 'get_booking_instructions', 'check_availability', 'make_reservation']
Created LangChain tool: search_restaurants
Created LangChain tool: get_restaurant_details
Created LangChain tool: get_booking_instructions
Created LangChain tool: check_availability
Created LangChain tool: make_reservation
✅ Agent created with 5 tools
Available tools: ['search_restaurants', 'get_restaurant_details', 'get_booking_instructions', 'check_availability', 'make_reservation']


In [4]:
# Enabling tracing for LangGraph (LangChain)
mlflow.langchain.autolog()

# Test with agent - this should now work without pydantic errors
response = agent.chat("Find me romantic Italian restaurants near Laguna Beach, CA for tonight")
print(f"Claude: {response}")

Debug: Calling MCP tool search_restaurants with args: {'placeName': 'Laguna Beach, CA', 'cuisineTypes': ['Italian'], 'mood': 'romantic', 'event': 'dating'}
Claude: I've found several romantic restaurants in Laguna Beach that would be perfect for tonight. Let me highlight the best options:

1. Nick's Laguna Beach
- Rating: 4.7/5 (2,400 reviews)
- Address: 440 S Coast Hwy, Laguna Beach
- Price Level: Moderate
- Currently open
- Takes reservations

2. Romeo Cucina
- Rating: 4.5/5 (680 reviews)
- Address: 249 Broadway St, Laguna Beach
- Price Level: Moderate
- Takes reservations
- Authentic Italian cuisine

3. The Cliff Restaurant
- Rating: 4.4/5 (4,289 reviews)
- Address: 577 S Coast Hwy, Laguna Beach
- Price Level: Moderate
- Currently open
- Beautiful ocean views

Would you like me to check availability at any of these restaurants for tonight? If so, I'll need:
1. What time you'd like to dine
2. How many people will be in your party

Once you provide these details, I can check availabil

In [7]:
# Run test scenarios
print("🧪 Running traced test scenarios\n")

test_threads = [
  [
    "Find me romantic Italian restaurants near Laguna Beach, CA for tonight",
    "What's the address of the first restaurant?",
    "Can you book a table for two for 7 PM?"
  ],
  [
    "I need casual Japanese restaurants near Irvine, CA for lunch",
    "What is the price range for the second restaurant?",
    "Can you book a table for four at the second restaurant for 12:30pm?",
  ],
  [
    "What do you think about Taiwanese street food?",
    "Can you recommend a good Taiwanese street food place near Costa Mesa, CA?",
    "Do they offer takeout?",
    "Can you check if they have vegetarian options?",
    "Can I order online?",
  ]
]

for thread_idx, conversation in enumerate(test_threads, 1):
    print(f"\n{'='*60}")
    print(f"Test Thread {thread_idx}:")

    for i, user_input in enumerate(conversation, 1):
        print(f"User Input {i}: {user_input}")
        try:
            response = agent.chat(user_input, f"test_thread_{thread_idx}")
            print(f"Claude Response: {response}")
        except Exception as e:
            print(f"Error: {e}")

print(f"\n🎯 Completed {len(test_cases)} test cases")

🧪 Running traced test scenarios


Test Thread 1:
User Input 1: Find me romantic Italian restaurants near Laguna Beach, CA for tonight
Debug: Calling MCP tool search_restaurants with args: {'placeName': 'Laguna Beach, CA', 'cuisineTypes': ['Italian'], 'mood': 'romantic', 'event': 'dating'}
Debug: Calling MCP tool get_restaurant_details with args: {'placeId': 'ChIJX5-mVDXk3IAR9rCbdYCI56o'}
Claude Response: Romeo Cucina would be an excellent choice for an Italian romantic dinner. Would you like me to help you check availability for tonight? If so, I'll need to know:
1. What time you'd like to dine
2. How many people will be in your party

Just let me know your preferences and I'll be happy to help you make a reservation!
User Input 2: What's the address of the first restaurant?
Claude Response: Nick's Laguna Beach is located at 440 S Coast Hwy, Laguna Beach, CA 92651, USA.

Would you like me to check availability or get more details about this restaurant for you?
User Input 3: Can you boo